# Compute the time slices and GWLs for soil moisture CMIP6 (data provided by Mathias Hauser)

### Author: Jerome Servonat
### Contact: jerome.servonnat at lsce.ipsl.fr

In [1]:
import xarray as xr
import glob, os
import numpy as np
import csv

### 0. Get region long and short names (abbrevs)

In [2]:
def retrieve_AR6regions():

    regions_filename='/home/jservon/Chapter12_IPCC/scripts/ATLAS/reference-regions/IPCC-WGI-reference-regions-v4_coordinates.csv'

    # -- Store the informations by region in the 'regions' dictionary
    regions = dict()
    with open(regions_filename) as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')#, quotechar='|')
        for row in spamreader:
            #if row[0]==region_name:
            print row[2]
            regions[row[2]]=row[3]
    #
    return regions
regions = retrieve_AR6regions()

Greenland/Iceland
N.W.North-America
N.E.North-America
W.North-America
C.North-America
E.North-America
N.Central-America
S.Central-America
Caribbean
N.W.South-America
N.South-America
N.E.South-America
South-American-Monsoon
S.W.South-America
S.E.South-America
S.South-America
N.Europe
West&Central-Europe
E.Europe
Mediterranean
Sahara
Western-Africa
Central-Africa
N.Eastern-Africa
S.Eastern-Africa
W.Southern-Africa
E.Southern-Africa
Madagascar
Russian-Arctic
Russian-Arctic
W.Siberia
E.Siberia
Russian-Far-East
W.C.Asia
E.C.Asia
Tibetan-Plateau
E.Asia
Arabian-Peninsula
S.Asia
S.E.Asia
N.Australia
C.Australia
E.Australia
S.Australia
New-Zealand
E.Antarctica
W.Antarctica
Arctic-Ocean
N.Pacific-Ocean
N.Pacific-Ocean
Equatorial.Pacific-Ocean
Equatorial.Pacific-Ocean
S.Pacific-Ocean
S.Pacific-Ocean
N.Atlantic-Ocean
Equatorial.Atlantic-Ocean
S.Atlantic-Ocean
Arabian-Sea
Bay-of-Bengal
Equatorial.Indic-Ocean
S.Indic-Ocean
Southern-Ocean


### We already have the regional averages precomputed by Mathias Hauser.
### We need to:
- extract the periods and compute the time average
- gather the results for each clim period of all the models for each region

## 1. Time averages for baseline, ssp126 mid/far and ssp585 mid/far

In [3]:
# -- List all the files and retrieve the models and realization
# -- Extract the period and compute the average => cdo timavg -seldate
# -- 
exp_list = [
    dict(experiment='historical',
         period = '1995-2014'),
    dict(experiment='ssp585',
         period = '2041-2060'),
    dict(experiment='ssp585',
         period = '2081-2100'),
    dict(experiment='ssp126',
         period = '2081-2100'),
    dict(experiment='ssp126',
         period = '2041-2060'),
]

In [4]:
models_per_exp = dict()
regional_averages = dict()
regional_averages_for_diff = dict()
for exp in exp_list:
    
    scenario = exp['experiment']
    period = exp['period']
    start_year = period.split('-')[0]
    end_year = period.split('-')[1]
    clim_period = scenario+'_'+period
    
    models_per_exp[clim_period] = dict()
    #regional_averages[clim_period] = dict()
    #regional_averages_for_diff[clim_period] = dict()
    
    #for long_region_name  in regions:
    #    regional_averages[clim_period][regions[long_region_name]] = []
    #    regional_averages_for_diff[clim_period][regions[long_region_name]] = []
    #
    # -- Get the list of files for the experiment / clim_period
    search_pattern = '/data/jservon/IPCC/mrso/tmp_20210225/data.iac.ethz.ch/IPCC_AR6/for_ch12/cmip6/mrso/sm_annmean_reg_ave_ar6/sm_annmean_reg_ave_ar6_mrso_Lmon_*_'+scenario+'_*.nc'
    lof = glob.glob(search_pattern)
    list_of_models = []
    list_of_files = []
    for wfile in lof:

        # -- Extract period and compute average
        outdir = '/data/jservon/IPCC/mrso/averages_over_periods/'
        # -- Create the output filename
        ncfilename = os.path.basename(wfile)
        outfilename = outdir+ncfilename.replace('.nc','_clim-'+start_year+'-'+end_year+'.nc')
        dum = ncfilename.split('_')
        wmodel = dum[7]
        models_per_exp[clim_period][wmodel] = outfilename
        
        # -- Select the period and compute the time average with cdo
        cmd = 'cdo timavg -selyear,'+start_year+'/'+end_year+' '+wfile+' '+outfilename
        #os.system(cmd)

In [5]:
models_per_exp['historical_1995-2014'].keys()

['CNRM-CM6-1-HR',
 'MPI-ESM1-2-HR',
 'UKESM1-0-LL',
 'HadGEM3-GC31-LL',
 'ACCESS-ESM1-5',
 'MPI-ESM-1-2-HAM',
 'ACCESS-CM2',
 'MPI-ESM1-2-LR',
 'CMCC-CM2-HR4',
 'MRI-ESM2-0',
 'NorESM2-LM',
 'IPSL-CM6A-LR',
 'CIESM',
 'CAMS-CSM1-0',
 'EC-Earth3-AerChem',
 'GISS-E2-1-G',
 'IPSL-CM6A-LR-INCA',
 'CanESM5',
 'CESM2-WACCM-FV2',
 'INM-CM4-8',
 'AWI-ESM-1-1-LR',
 'GISS-E2-1-G-CC',
 'INM-CM5-0',
 'CAS-ESM2-0',
 'CNRM-ESM2-1',
 'CNRM-CM6-1',
 'GISS-E2-1-H',
 'CMCC-CM2-SR5',
 'CESM2-WACCM',
 'CMCC-ESM2',
 'SAM0-UNICON',
 'E3SM-1-1-ECA',
 'CESM2',
 'TaiESM1',
 'EC-Earth3-Veg',
 'CESM2-FV2',
 'NorESM2-MM',
 'FGOALS-f3-L',
 'EC-Earth3-Veg-LR',
 'GFDL-CM4',
 'EC-Earth3',
 'BCC-CSM2-MR',
 'EC-Earth3-CC',
 'CanESM5-CanOE',
 'MIROC6',
 'MCM-UA-1-0',
 'HadGEM3-GC31-MM',
 'GFDL-ESM4',
 'BCC-ESM1',
 'E3SM-1-1',
 'E3SM-1-0',
 'FGOALS-g3',
 'MIROC-ES2L']

In [6]:
# Pour chaque scenario futur, il me faut l'équivalent historical

# Boucle sur les scenarios futurs

In [7]:
# Sauvegarder les resultats en bruts en difference

## Raw values, all models available per time slice

In [8]:
for exp in ['historical_1995-2014',
            'ssp126_2041-2060',
            'ssp126_2081-2100',
            'ssp585_2041-2060',
            'ssp585_2081-2100']:
    regional_averages[exp] = dict()
    for long_region_name in regions:
        regional_averages[exp][regions[long_region_name]] = []
    # -- For each future clim period, we loop on all the models availables
    for model in models_per_exp[exp]:
        # -- Raw values
        # ------------------------------------------------------------------
        # -- Get the data with Xarray for the future time slice
        wfile_exp = models_per_exp[exp][model]
        dum = xr.open_dataset(wfile_exp)
        dat = dum['mrso']
        region_names = list(dum['names'].values)

        # -- Loop on the regional averages available in the file
        for region_name in region_names:
            if region_name in regions:
                shortname = regions[region_name]
                ind = region_names.index(region_name)
                value = float(dat[0][ind].values)
                regional_averages[exp][shortname].append( value )            


In [9]:
## Gather the data to compute the differences model by model

In [10]:
# -- Loop on the time periods
for exp in ['ssp126_2041-2060', 'ssp126_2081-2100','ssp585_2041-2060','ssp585_2081-2100']:
    regional_averages_for_diff[exp] = dict()
    regional_averages_for_diff[exp+'_historical_1995-2014'] = dict()
    for long_region_name in regions:
        regional_averages_for_diff[exp][regions[long_region_name]] = []
        regional_averages_for_diff[exp+'_historical_1995-2014'][regions[long_region_name]] = []


    # -- Differences
    # ------------------------------------------------------------------
    # -- For each future clim period, we loop on all the models availables
    for model in models_per_exp[exp]:
        # -- And if the model is also available for baseline, we keep going
        if model in models_per_exp['historical_1995-2014']:
            wfile_baseline = models_per_exp['historical_1995-2014'][model]
            wfile_exp = models_per_exp[exp][model]

            # -- Get the data with Xarray for the future time slice
            dum = xr.open_dataset(wfile_exp)
            dat = dum['mrso']
            region_names = list(dum['names'].values)

            # -- Loop on the regional averages available in the file
            for region_name in region_names:
                if region_name in regions:
                    shortname = regions[region_name]
                    ind = region_names.index(region_name)
                    value = float(dat[0][ind].values)
                    regional_averages_for_diff[exp][shortname].append( value )            

            # -- Get the data for the baseline
            dum = xr.open_dataset(wfile_baseline)
            dat = dum['mrso']
            region_names = list(dum['names'].values)

            # -- Short name
            for region_name in region_names:
                if region_name in regions:
                    shortname = regions[region_name]
                    ind = region_names.index(region_name)
                    value = float(dat[0][ind].values)
                    regional_averages_for_diff[exp+'_historical_1995-2014'][shortname].append( value )            


In [7]:
regional_averages_for_diff['ssp126_2041-2060_historical_1995-2014']

{'ARO': [],
 'ARP': [138.3467146702001,
  176.63681591670127,
  319.19785257460626,
  660.0003182849089,
  35.11562956046852,
  93.75070003505105,
  46.10455946073505,
  1193.3873106150475,
  307.1016456163073,
  501.63805815996795,
  132.52455342742428,
  128.20104084391792,
  619.9440829421507,
  261.32819443768034,
  133.82419126997078,
  137.403653943186,
  634.4968225296413,
  1275.2792668525517,
  393.9051559123326,
  319.57546294987293,
  409.77103321641255,
  1312.2231363638314,
  639.4197082520459,
  534.5434322990895,
  1222.5550306790187,
  743.7621986472886,
  33.38322615091592,
  39.39361169754205,
  324.8308833569248,
  573.9799776865409,
  2025.3191016316614,
  10.184911662779784,
  1967.7151419336647,
  130.21038613661486,
  2018.7289949809892],
 'ARS': [],
 'BOB': [],
 'CAF': [1011.1910798160554,
  789.4489363462495,
  662.5240668683324,
  1098.0374059428868,
  822.5044525462496,
  814.8393270682074,
  598.2372154973477,
  1960.414160984632,
  476.42506863773434,
  893

## 2. GWLs 1.5, 2, 3 and 4

In [11]:
GWL_csv = '/home/jservon/Chapter12_IPCC/scripts/ATLAS/warming-levels/CMIP6_Atlas_WarmingLevels.csv'

GWL_dict = dict()
i = 0
with open(GWL_csv) as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')#, quotechar='|')
    for row in spamreader:
        print row
        model = row[0]#.split('_')[0]
        print model
        GWL_dict[model] = dict()
        if i==0:
            colnames = row
        j = 1
        for elt in row[1:len(row)]:
            print elt
            GWL_dict[model][colnames[j]] = row[j]
            j = j + 1
        i = i + 1

['', '1.5_ssp126', '2_ssp126', '3_ssp126', '4_ssp126', '1.5_ssp245', '2_ssp245', '3_ssp245', '4_ssp245', '1.5_ssp370', '2_ssp370', '3_ssp370', '4_ssp370', '1.5_ssp585', '2_ssp585', '3_ssp585', '4_ssp585']

1.5_ssp126
2_ssp126
3_ssp126
4_ssp126
1.5_ssp245
2_ssp245
3_ssp245
4_ssp245
1.5_ssp370
2_ssp370
3_ssp370
4_ssp370
1.5_ssp585
2_ssp585
3_ssp585
4_ssp585
['ACCESS-CM2_r1i1p1f1', '2027', '2042', 'NA', 'NA', '2028', '2040', '2070', 'NA', '2027', '2039', '2062', '2082', '2025', '2038', '2055', '2070']
ACCESS-CM2_r1i1p1f1
2027
2042
NA
NA
2028
2040
2070
NA
2027
2039
2062
2082
2025
2038
2055
2070
['ACCESS-ESM1-5_r1i1p1f1', '2030', '2072', 'NA', 'NA', '2029', '2045', 'NA', 'NA', '2033', '2048', '2069', 'NA', '2027', '2038', '2060', '2078']
ACCESS-ESM1-5_r1i1p1f1
2030
2072
NA
NA
2029
2045
NA
NA
2033
2048
2069
NA
2027
2038
2060
2078
['AWI-CM-1-1-MR_r1i1p1f1', '2022', '2050', 'NA', 'NA', '2020', '2039', 'NA', 'NA', '2021', '2037', '2064', 'NA', '2019', '2036', '2059', '2079']
AWI-CM-1-1-MR_r1i1p

In [12]:
list_of_GWLs = ['1.5','2','3','4']

#files_per_GWL = dict()
#for GWL in ['1.5','2']:
#    files_per_GWL[GWL] = dict()
req_dict = dict(project='CMIP6',
                table='Amon'
               )

models_per_GWL = dict()
for GWL in list_of_GWLs:
    models_per_GWL[GWL] = dict()
    #regional_averages[GWL] = dict()
    #regional_averages[GWL+'_historical_1995-2014'] = dict()
    #for long_region_name  in regions:
    #    regional_averages[GWL][regions[long_region_name]] = []
    #    #regional_averages[GWL+'_historical_1995-2014'][regions[long_region_name]] = []

        
#
#Pour Chaque GWL, je garde le model_realization_scenario dans lequel je mets le path vers le fichier
for scenario in ['26','85']:

    if scenario=='26': req_scenario = 'ssp126'
    if scenario=='85': req_scenario = 'ssp585'

    search_pattern = '/data/jservon/IPCC/mrso/tmp_20210225/data.iac.ethz.ch/IPCC_AR6/for_ch12/cmip6/mrso/sm_annmean_reg_ave_ar6/sm_annmean_reg_ave_ar6_mrso_Lmon_*_'+req_scenario+'_*.nc'
    lof = glob.glob(search_pattern)
    for wfile in lof:
        dum = os.path.basename(wfile).split('_')
        wmodel = dum[7]
        wrealization = dum[9]
        print wmodel, wrealization
        wmodel_realization = wmodel+'_'+wrealization
        if wmodel_realization in GWL_dict:
            print 'We have : ', wmodel_realization
            print GWL_dict[wmodel_realization]
            for GWL in list_of_GWLs:
                if scenario=='26': GWL_scenario = GWL+'_ssp126'
                if scenario=='85': GWL_scenario = GWL+'_ssp585'

                # --> file nc
                # --> period
                central_year = GWL_dict[wmodel_realization][GWL_scenario]
                if central_year not in ['NA','9999']:
                    start_year = str( int(central_year)-9 )
                    end_year = str( int(central_year)+10 )
                #
                ncfilename = os.path.basename(wfile)
                outfilename = outdir+ncfilename.replace('.nc','_clim-GWL'+GWL+'.nc')
                cmd = 'cdo timavg -selyear,'+start_year+'/'+end_year+' '+wfile+' '+outfilename
                #os.system(cmd)
                
                models_per_GWL[GWL][wmodel_realization+'_'+scenario] = outfilename
                

NorESM2-MM r1i1p1f1
We have :  NorESM2-MM_r1i1p1f1
{'2_ssp126': 'NA', '2_ssp585': '2054', '4_ssp126': 'NA', '3_ssp585': '2076', '1.5_ssp245': '2046', '1.5_ssp370': '2046', '1.5_ssp126': 'NA', '1.5_ssp585': '2039', '3_ssp370': 'NA', '4_ssp585': 'NA', '4_ssp370': 'NA', '2_ssp245': '2077', '3_ssp245': 'NA', '2_ssp370': '2062', '3_ssp126': 'NA', '4_ssp245': 'NA'}
IPSL-CM6A-LR r1i1p1f1
We have :  IPSL-CM6A-LR_r1i1p1f1
{'2_ssp126': '2038', '2_ssp585': '2034', '4_ssp126': 'NA', '3_ssp585': '2050', '1.5_ssp245': '2018', '1.5_ssp370': '2019', '1.5_ssp126': '2019', '1.5_ssp585': '2018', '3_ssp370': '2055', '4_ssp585': '2066', '4_ssp370': '2076', '2_ssp245': '2033', '3_ssp245': '2064', '2_ssp370': '2034', '3_ssp126': 'NA', '4_ssp245': 'NA'}
MPI-ESM1-2-LR r1i1p1f1
We have :  MPI-ESM1-2-LR_r1i1p1f1
{'2_ssp126': 'NA', '2_ssp585': '2048', '4_ssp126': 'NA', '3_ssp585': '2071', '1.5_ssp245': '2036', '1.5_ssp370': '2035', '1.5_ssp126': '2042', '1.5_ssp585': '2034', '3_ssp370': '2078', '4_ssp585': 'NA', 

## Raw values of GWLs

In [13]:
for GWL in list_of_GWLs:
    regional_averages[GWL] = dict()
    for long_region_name in regions:
        regional_averages[GWL][regions[long_region_name]] = []
    # -- For each future clim period, we loop on all the models availables
    for model in models_per_GWL[GWL]:
        # -- Raw values
        # ------------------------------------------------------------------
        # -- Get the data with Xarray for the future time slice
        wfile_exp = models_per_GWL[GWL][model]
        dum = xr.open_dataset(wfile_exp)
        dat = dum['mrso']
        region_names = list(dum['names'].values)

        # -- Loop on the regional averages available in the file
        for region_name in region_names:
            if region_name in regions:
                shortname = regions[region_name]
                ind = region_names.index(region_name)
                value = float(dat[0][ind].values)
                regional_averages[GWL][shortname].append( value )            


## Save results to compute the differences

In [14]:
for GWL in list_of_GWLs:
    print 'GWL = ', GWL
    regional_averages_for_diff[GWL] = dict()
    regional_averages_for_diff[GWL+'_historical_1995-2014'] = dict()
    for long_region_name in regions:
        regional_averages_for_diff[GWL][regions[long_region_name]] = []
        regional_averages_for_diff[GWL+'_historical_1995-2014'][regions[long_region_name]] = []
    
    for model in models_per_GWL[GWL]:
        wmodel = model.split('_')[0]
        if wmodel in models_per_exp['historical_1995-2014']:
            wfile_baseline = models_per_exp['historical_1995-2014'][wmodel]
            wfile_exp = models_per_GWL[GWL][model]
            
            # -- Get the data with Xarray
            dum = xr.open_dataset(wfile_exp)
            dat = dum['mrso']
            region_names = list(dum['names'].values)

            # -- Short name
            for region_name in region_names:
                if region_name in regions:
                    shortname = regions[region_name]
                    ind = region_names.index(region_name)
                    value = float(dat[0][ind].values)
                    regional_averages_for_diff[GWL][shortname].append( value )            

            # -- Get the data with Xarray
            dum = xr.open_dataset(wfile_baseline)
            dat = dum['mrso']
            region_names = list(dum['names'].values)

            # -- Short name
            for region_name in region_names:
                if region_name in regions:
                    shortname = regions[region_name]
                    ind = region_names.index(region_name)
                    value = float(dat[0][ind].values)
                    regional_averages_for_diff[GWL+'_historical_1995-2014'][shortname].append( value )            


GWL =  1.5
GWL =  2
GWL =  3
GWL =  4


## 3. Compute the percentiles and save in a json file

In [13]:
quantiles_dict = dict()
for clim_period in regional_averages:
    quantiles_dict[clim_period] = dict()
    for region_name in regional_averages[clim_period]:
        print clim_period, region_name
        if regional_averages[clim_period][region_name]:
            dat = np.array(regional_averages[clim_period][region_name])
            q10 = np.quantile(dat, 0.1)
            q50 = np.quantile(dat, 0.5)
            q90 = np.quantile(dat, 0.9)
            quantiles_dict[clim_period][region_name] = [q10, q50, q90]
import json
ensemble = 'CMIP6'
outfilename = '/home/jservon/Chapter12_IPCC/data/SM_satellites/'+ensemble+'_SM_AR6_regional_averages.json'
#print outfilename
with open(outfilename, 'w') as fp:
    json.dump(quantiles_dict, fp, sort_keys=True, indent=4)

1.5 NEN
1.5 BOB
1.5 WSB
1.5 ENA
1.5 EEU
1.5 CAR
1.5 WSAF
1.5 NAO
1.5 CAU
1.5 SPO*
1.5 NAU
1.5 EPO*
1.5 SWS
1.5 NEU
1.5 SIO
1.5 SEAF
1.5 TIB
1.5 WNA
1.5 SES
1.5 ECA
1.5 NWS
1.5 NSA
1.5 SCA
1.5 MDG
1.5 SOO
1.5 NES
1.5 SAM
1.5 NZ
1.5 EIO
1.5 SEA
1.5 SAS
1.5 NWN
1.5 EAU
1.5 GIC
1.5 SAU
1.5 WAN
1.5 WAF
1.5 RAR*
1.5 NPO*
1.5 WCA
1.5 SAH
1.5 WCE
1.5 ESB
1.5 RFE
1.5 CNA
1.5 SAO
1.5 EAS
1.5 ARP
1.5 MED
1.5 ARS
1.5 EAO
1.5 NEAF
1.5 EAN
1.5 CAF
1.5 NCA
1.5 SSA
1.5 ESAF
1.5 ARO
ssp126_2081-2100 NEN
ssp126_2081-2100 BOB
ssp126_2081-2100 WSB
ssp126_2081-2100 ENA
ssp126_2081-2100 EEU
ssp126_2081-2100 CAR
ssp126_2081-2100 WSAF
ssp126_2081-2100 NAO
ssp126_2081-2100 CAU
ssp126_2081-2100 SPO*
ssp126_2081-2100 NAU
ssp126_2081-2100 EPO*
ssp126_2081-2100 SWS
ssp126_2081-2100 NEU
ssp126_2081-2100 SIO
ssp126_2081-2100 SEAF
ssp126_2081-2100 TIB
ssp126_2081-2100 WNA
ssp126_2081-2100 SES
ssp126_2081-2100 ECA
ssp126_2081-2100 NWS
ssp126_2081-2100 NSA
ssp126_2081-2100 SCA
ssp126_2081-2100 MDG
ssp126_2081-2100 SOO


In [14]:
quantiles_dict = dict()
for clim_period in regional_averages_for_diff:
    quantiles_dict[clim_period] = dict()
    for region_name in regional_averages_for_diff[clim_period]:
        print clim_period, region_name
        if regional_averages_for_diff[clim_period][region_name]:
            if 'historical_1995-2014' not in clim_period:
                baseline = np.array(regional_averages_for_diff[clim_period+'_historical_1995-2014'][region_name])
                dat = np.array(regional_averages_for_diff[clim_period][region_name])
                q10 = np.quantile(dat-baseline, 0.1)
                q50 = np.quantile(dat-baseline, 0.5)
                q90 = np.quantile(dat-baseline, 0.9)
                quantiles_dict[clim_period][region_name] = [q10, q50, q90]
import json
ensemble = 'CMIP6'
outfilename = '/home/jservon/Chapter12_IPCC/data/SM_satellites/'+ensemble+'_SM_diff_AR6_regional_averages.json'
#print outfilename
with open(outfilename, 'w') as fp:
    json.dump(quantiles_dict, fp, sort_keys=True, indent=4)

1.5 NEN
1.5 BOB
1.5 WSB
1.5 ENA
1.5 EEU
1.5 CAR
1.5 WSAF
1.5 NAO
1.5 CAU
1.5 SPO*
1.5 NAU
1.5 EPO*
1.5 SWS
1.5 NEU
1.5 SIO
1.5 SEAF
1.5 TIB
1.5 WNA
1.5 SES
1.5 ECA
1.5 NWS
1.5 NSA
1.5 SCA
1.5 MDG
1.5 SOO
1.5 NES
1.5 SAM
1.5 NZ
1.5 EIO
1.5 SEA
1.5 SAS
1.5 NWN
1.5 EAU
1.5 GIC
1.5 SAU
1.5 WAN
1.5 WAF
1.5 RAR*
1.5 NPO*
1.5 WCA
1.5 SAH
1.5 WCE
1.5 ESB
1.5 RFE
1.5 CNA
1.5 SAO
1.5 EAS
1.5 ARP
1.5 MED
1.5 ARS
1.5 EAO
1.5 NEAF
1.5 EAN
1.5 CAF
1.5 NCA
1.5 SSA
1.5 ESAF
1.5 ARO
ssp585_2041-2060_historical_1995-2014 NEN
ssp585_2041-2060_historical_1995-2014 BOB
ssp585_2041-2060_historical_1995-2014 WSB
ssp585_2041-2060_historical_1995-2014 ENA
ssp585_2041-2060_historical_1995-2014 EEU
ssp585_2041-2060_historical_1995-2014 CAR
ssp585_2041-2060_historical_1995-2014 WSAF
ssp585_2041-2060_historical_1995-2014 NAO
ssp585_2041-2060_historical_1995-2014 CAU
ssp585_2041-2060_historical_1995-2014 SPO*
ssp585_2041-2060_historical_1995-2014 NAU
ssp585_2041-2060_historical_1995-2014 EPO*
ssp585_2041-2060_histo

ssp585_2081-2100 MED
ssp585_2081-2100 ARS
ssp585_2081-2100 EAO
ssp585_2081-2100 NEAF
ssp585_2081-2100 EAN
ssp585_2081-2100 CAF
ssp585_2081-2100 NCA
ssp585_2081-2100 SSA
ssp585_2081-2100 ESAF
ssp585_2081-2100 ARO
ssp126_2081-2100_historical_1995-2014 NEN
ssp126_2081-2100_historical_1995-2014 BOB
ssp126_2081-2100_historical_1995-2014 WSB
ssp126_2081-2100_historical_1995-2014 ENA
ssp126_2081-2100_historical_1995-2014 EEU
ssp126_2081-2100_historical_1995-2014 CAR
ssp126_2081-2100_historical_1995-2014 WSAF
ssp126_2081-2100_historical_1995-2014 NAO
ssp126_2081-2100_historical_1995-2014 CAU
ssp126_2081-2100_historical_1995-2014 SPO*
ssp126_2081-2100_historical_1995-2014 NAU
ssp126_2081-2100_historical_1995-2014 EPO*
ssp126_2081-2100_historical_1995-2014 SWS
ssp126_2081-2100_historical_1995-2014 NEU
ssp126_2081-2100_historical_1995-2014 SIO
ssp126_2081-2100_historical_1995-2014 SEAF
ssp126_2081-2100_historical_1995-2014 TIB
ssp126_2081-2100_historical_1995-2014 WNA
ssp126_2081-2100_historical_

ssp126_2041-2060 NCA
ssp126_2041-2060 SSA
ssp126_2041-2060 ESAF
ssp126_2041-2060 ARO
ssp585_2081-2100_historical_1995-2014 NEN
ssp585_2081-2100_historical_1995-2014 BOB
ssp585_2081-2100_historical_1995-2014 WSB
ssp585_2081-2100_historical_1995-2014 ENA
ssp585_2081-2100_historical_1995-2014 EEU
ssp585_2081-2100_historical_1995-2014 CAR
ssp585_2081-2100_historical_1995-2014 WSAF
ssp585_2081-2100_historical_1995-2014 NAO
ssp585_2081-2100_historical_1995-2014 CAU
ssp585_2081-2100_historical_1995-2014 SPO*
ssp585_2081-2100_historical_1995-2014 NAU
ssp585_2081-2100_historical_1995-2014 EPO*
ssp585_2081-2100_historical_1995-2014 SWS
ssp585_2081-2100_historical_1995-2014 NEU
ssp585_2081-2100_historical_1995-2014 SIO
ssp585_2081-2100_historical_1995-2014 SEAF
ssp585_2081-2100_historical_1995-2014 TIB
ssp585_2081-2100_historical_1995-2014 WNA
ssp585_2081-2100_historical_1995-2014 SES
ssp585_2081-2100_historical_1995-2014 ECA
ssp585_2081-2100_historical_1995-2014 NWS
ssp585_2081-2100_historical_1

In [16]:
quantiles_dict = dict()
for clim_period in regional_averages_for_diff:
    quantiles_dict[clim_period] = dict()
    for region_name in regional_averages_for_diff[clim_period]:
        print clim_period, region_name
        if regional_averages_for_diff[clim_period][region_name]:
            if 'historical_1995-2014' not in clim_period:
                baseline = np.array(regional_averages_for_diff[clim_period+'_historical_1995-2014'][region_name])
                dat = np.array(regional_averages_for_diff[clim_period][region_name])
                q10 = np.quantile(((dat-baseline)/baseline)*100, 0.1)
                q50 = np.quantile(((dat-baseline)/baseline)*100, 0.5)
                q90 = np.quantile(((dat-baseline)/baseline)*100, 0.9)
                quantiles_dict[clim_period][region_name] = [q10, q50, q90]
import json
ensemble = 'CMIP6'
outfilename = '/home/jservon/Chapter12_IPCC/data/SM_satellites/'+ensemble+'_SM_diff_perc2020_AR6_regional_averages.json'
#print outfilename
with open(outfilename, 'w') as fp:
    json.dump(quantiles_dict, fp, sort_keys=True, indent=4)

1.5 NEN
1.5 BOB
1.5 WSB
1.5 ENA
1.5 EEU
1.5 CAR
1.5 WSAF
1.5 NAO
1.5 CAU
1.5 SPO*
1.5 NAU
1.5 EPO*
1.5 SWS
1.5 NEU
1.5 SIO
1.5 SEAF
1.5 TIB
1.5 WNA
1.5 SES
1.5 ECA
1.5 NWS
1.5 NSA
1.5 SCA
1.5 MDG
1.5 SOO
1.5 NES
1.5 SAM
1.5 NZ
1.5 EIO
1.5 SEA
1.5 SAS
1.5 NWN
1.5 EAU
1.5 GIC
1.5 SAU
1.5 WAN
1.5 WAF
1.5 RAR*
1.5 NPO*
1.5 WCA
1.5 SAH
1.5 WCE
1.5 ESB
1.5 RFE
1.5 CNA
1.5 SAO
1.5 EAS
1.5 ARP
1.5 MED
1.5 ARS
1.5 EAO
1.5 NEAF
1.5 EAN
1.5 CAF
1.5 NCA
1.5 SSA
1.5 ESAF
1.5 ARO
ssp585_2041-2060_historical_1995-2014 NEN
ssp585_2041-2060_historical_1995-2014 BOB
ssp585_2041-2060_historical_1995-2014 WSB
ssp585_2041-2060_historical_1995-2014 ENA
ssp585_2041-2060_historical_1995-2014 EEU
ssp585_2041-2060_historical_1995-2014 CAR
ssp585_2041-2060_historical_1995-2014 WSAF
ssp585_2041-2060_historical_1995-2014 NAO
ssp585_2041-2060_historical_1995-2014 CAU
ssp585_2041-2060_historical_1995-2014 SPO*
ssp585_2041-2060_historical_1995-2014 NAU
ssp585_2041-2060_historical_1995-2014 EPO*
ssp585_2041-2060_histo

/modfs/modtools/miniconda2/envs/analyse_2.7_v2/lib/python2.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in divide
  # Remove the CWD from sys.path while we load stuff.
/modfs/modtools/miniconda2/envs/analyse_2.7_v2/lib/python2.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in divide
  # This is added back by InteractiveShellApp.init_path()
/modfs/modtools/miniconda2/envs/analyse_2.7_v2/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in divide
  if sys.path[0] == '':


ssp126_2081-2100 TIB
ssp126_2081-2100 WNA
ssp126_2081-2100 SES
ssp126_2081-2100 ECA
ssp126_2081-2100 NWS
ssp126_2081-2100 NSA
ssp126_2081-2100 SCA
ssp126_2081-2100 MDG
ssp126_2081-2100 SOO
ssp126_2081-2100 NES
ssp126_2081-2100 SAM
ssp126_2081-2100 NZ
ssp126_2081-2100 EIO
ssp126_2081-2100 SEA
ssp126_2081-2100 SAS
ssp126_2081-2100 NWN
ssp126_2081-2100 EAU
ssp126_2081-2100 GIC
ssp126_2081-2100 SAU
ssp126_2081-2100 WAN
ssp126_2081-2100 WAF
ssp126_2081-2100 RAR*
ssp126_2081-2100 NPO*
ssp126_2081-2100 WCA
ssp126_2081-2100 SAH
ssp126_2081-2100 WCE
ssp126_2081-2100 ESB
ssp126_2081-2100 RFE
ssp126_2081-2100 CNA
ssp126_2081-2100 SAO
ssp126_2081-2100 EAS
ssp126_2081-2100 ARP
ssp126_2081-2100 MED
ssp126_2081-2100 ARS
ssp126_2081-2100 EAO
ssp126_2081-2100 NEAF
ssp126_2081-2100 EAN
ssp126_2081-2100 CAF
ssp126_2081-2100 NCA
ssp126_2081-2100 SSA
ssp126_2081-2100 ESAF
ssp126_2081-2100 ARO
2_historical_1995-2014 NEN
2_historical_1995-2014 BOB
2_historical_1995-2014 WSB
2_historical_1995-2014 ENA
2_histo

ssp585_2041-2060 SWS
ssp585_2041-2060 NEU
ssp585_2041-2060 SIO
ssp585_2041-2060 SEAF
ssp585_2041-2060 TIB
ssp585_2041-2060 WNA
ssp585_2041-2060 SES
ssp585_2041-2060 ECA
ssp585_2041-2060 NWS
ssp585_2041-2060 NSA
ssp585_2041-2060 SCA
ssp585_2041-2060 MDG
ssp585_2041-2060 SOO
ssp585_2041-2060 NES
ssp585_2041-2060 SAM
ssp585_2041-2060 NZ
ssp585_2041-2060 EIO
ssp585_2041-2060 SEA
ssp585_2041-2060 SAS
ssp585_2041-2060 NWN
ssp585_2041-2060 EAU
ssp585_2041-2060 GIC
ssp585_2041-2060 SAU
ssp585_2041-2060 WAN
ssp585_2041-2060 WAF
ssp585_2041-2060 RAR*
ssp585_2041-2060 NPO*
ssp585_2041-2060 WCA
ssp585_2041-2060 SAH
ssp585_2041-2060 WCE
ssp585_2041-2060 ESB
ssp585_2041-2060 RFE
ssp585_2041-2060 CNA
ssp585_2041-2060 SAO
ssp585_2041-2060 EAS
ssp585_2041-2060 ARP
ssp585_2041-2060 MED
ssp585_2041-2060 ARS
ssp585_2041-2060 EAO
ssp585_2041-2060 NEAF
ssp585_2041-2060 EAN
ssp585_2041-2060 CAF
ssp585_2041-2060 NCA
ssp585_2041-2060 SSA
ssp585_2041-2060 ESAF
ssp585_2041-2060 ARO
4_historical_1995-2014 NEN
4_h